In [1]:
import pandas as pd
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
service = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(service=service, options=options)

In [3]:
url = 'https://www.transfermarkt.com/spieler-statistik/wertvollstespieler/marktwertetop'
driver.get(url)
time.sleep(3)

page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')

In [4]:
table = soup.find('table', class_ = 'items')
print(table.get_text())




#PlayerAgeNat.ClubMarket value



1Lamine YamalRight Winger18€200.00m 

2Erling HaalandCentre-Forward25€200.00m  

3Jude BellinghamAttacking Midfield22€180.00m 

4Kylian MbappéCentre-Forward26€180.00m 

5Vinicius JuniorLeft Winger25€150.00m 

6PedriCentral Midfield23€140.00m 

7Jamal MusialaAttacking Midfield22€140.00m 

8Michael OliseRight Winger24€130.00m 

9Bukayo SakaRight Winger24€130.00m  

10Federico ValverdeCentral Midfield27€130.00m 

11Cole PalmerAttacking Midfield23€120.00m  

12Declan RiceCentral Midfield26€120.00m  

13Alexander IsakCentre-Forward26€120.00m  

14Moisés CaicedoDefensive Midfield24€110.00m  

15Florian WirtzAttacking Midfield22€110.00m  

16Julián AlvarezCentre-Forward25€100.00m 

17Ousmane DembéléCentre-Forward28€100.00m 

18Désiré DouéRight Winger20€90.00m 

19João NevesCentral Midfield21€90.00m 

20Khvicha KvaratskheliaLeft Winger24€90.00m 

21William SalibaCentre-Back24€90.00m  

22VitinhaDefensive Midfield25€90.00m 

23Ryan GravenberchDefensive Midfi

In [16]:
data = []
rows = table.find('tbody').find_all('tr')
for row in rows:
    cols = row.find_all('td', recursive=False)
    if len(cols) < 6:
        continue
    # player info
    player_cell = cols[1]
    inner_table = player_cell.find('table', class_ = 'inline-table')
    if not inner_table:
        continue
    inner_rows = inner_table.find_all('tr')
    if len(inner_rows) < 2:
        continue
    name = inner_rows[0].find_all('td')[1].text.strip()
    pos = inner_rows[1].find('td').text.strip()
    
    # nationalities
    nat_imgs = cols[3].find_all('img')
    nat = [img.get('title', '') for img in nat_imgs]
    nat1 = nat[0] if len(nat) >= 1 else ''
    nat2 = nat[1] if len(nat) >= 2 else ''
    
    # club
    club_img = cols[4].find('img')
    club = club_img.get('title', '')
    
    # other data
    rank = cols[0].text.strip()
    age = cols[2].text.strip()
    mk_val = cols[-1].text.strip()
    
    data.append([rank, name, pos, age, nat1, nat2, club, mk_val])

columns = [
    'Rank',
    'Name',
    'Position',
    'Age',
    'Nationality_1',
    'Nationality_2',
    'Club',
    'Maket_value_EUR'
]

df = pd.DataFrame(data, columns=columns)

In [18]:
df.to_csv('scraping_tranfermkt_selenium.v1.csv', index=False, encoding='utf-8-sig')